In [2]:
import numpy as np

class Decoder(object):
    def __init__(self, initialProb, transProb, obsProb):
        self.N = initialProb.shape[0]
        self.initialProb = initialProb
        self.transProb = transProb
        self.obsProb = obsProb
        assert self.initialProb.shape == (self.N, 1)
        assert self.transProb.shape == (self.N, self.N)
        assert self.obsProb.shape[0] == self.N
        
    def Obs(self, obs):
        return self.obsProb[:, obs, None]

    def Decode(self, obs):
        trellis = np.zeros((self.N, len(obs)))
        backpt = np.ones((self.N, len(obs)), 'int32') * -1
                
        # initialization
        trellis[:, 0] = np.squeeze(self.initialProb * self.Obs(obs[0]))
                
        for t in range(1, len(obs)):
            trellis[:, t] = (trellis[:, t-1, None].dot(self.Obs(obs[t]).T) * self.transProb).max(0)
            backpt[:, t] = (np.tile(trellis[:, t-1, None], [1, self.N]) * self.transProb).argmax(0)
        # termination
        tokens = [trellis[:, -1].argmax()]
        for i in range(len(obs)-1, 0, -1):
            tokens.append(backpt[tokens[-1], i])
        return tokens[::-1]

In [13]:
W=0
X=1
Y=2
Z=3

pi = np.array([[0.32, 0.28, 0.4 ]]).T
trans = np.array([ \
    [0, 0.2, 0.8], \
    [0.7, 0.1, 0.2], \
    [0.3, 0.6, 0.1]])
obs = np.array([[0.6,0.1, 0.2, 0.1], \
                [0.1, 0.7, 0.1, 0.1], \
                [0.2, 0, 0.5, 0.3]])
# Test
data_test = [X,W,Z,Y,W,Z,X,W,X,W]

# Data Sets
data_1 = [Z,Y,X,Z,W,W,Y,X,Y,X]
data_2 = [X,W,X,Y,Z,X,Y,Y,X,W,X,Z,Y,X,X,W,X,Y,Z,Z,X,Z,X,Z,Y]
data_3 = [Y,X,W,Y,X,X,Z,Y,Y,Y,Z,W,Y,W,X,W,Z,X,Z,Y,X,Z,W,Z,Z,Y,W,Z,Z,W,Y,Y,Y,W,W,Y,W,X,X,
          W,X,Z,X,Y,X,W,Z,X,Y,X,W,X,Y,X,W,Y,X,W,W,W,X,W,Y,X,Y,W,Y,Y,X,Y,Z,Y,Y,X,X,Y,Y,W,
          Y,X,W,X,W,Z,X,W,Z,Z,X,W,Y,X,Y,X,Y,Z,W,W,Y,W]
data_4 = [W,Y,Y,Y,Z,X,X,W,Z,X,Y,X,Z,Y,X,X,Y,W,X,W,Y,Z,Z,X,W,Z,X,W,Y,Z,X,W,W,X,Z,Z,X,Y,Y,W,Z,X,Y,X,Z,X,W,Y,X,W,Z,X,W,W,Z,X,W,Y,Z,Y,Z,X,X,Z,X,Y,Z,Z,Y,X,X,Y,X,Z,Y,X,W,Y,Z,X,W,Y,X,X,W,Z,Y,Z,X,Z,W,Y,X,W,Y,X,W,W,X,Z,Y,W,Y,Z,Y,Y,X,Z,Y,X,W,Y,W,X,Y,Y,X,Z,Z,X,W,Y,X,X,W,Z,X,X,Y,X,Z,Y,X,X,W,Y,Z,X,Z,Y,Z,X,X,W,X,W,Y,W,X,W,Y,X,W,W,X,X,X,W,X,W,X,W,Y,W,X,W,W,Y,X,Y,Y,X,W,Y,X,Y,Z,W,W,X,W,W,W,X,Z,X,W,Z,Z,Y,Y,Z,X,X,X,W,W,X,Y,Y,Y,Y,Y,X,Y,Z,X,X,W,Z,W,Z,X,W,Y,W,X,W,W,W,Y,X,Y,X,W,W,Z,Y,Y,W,W,Y,Z,X,W,Y,W,W,Z,Z,Y,W,W,X,Y,Y,X,X,Y,Y,W,Y,X,W,Y,Y,Z,W,X,W,Z,Y,X,Z,X,W,Z,X,W,Y,W,Y,W,Y,X,W,Z,X,X,Y,Z,X,Z,Y,X,W,Y,X,Z,Y,Z,W,Z,X,X,W,Z,W,W,X,W,X,Y,X,W,Y,X,W,Z,W,W,W,W,Y,W,Y,Y,W,Y,X,X,Y,X,W,W,W,Y,Y,W,X,Y,Y,X,W,X,W,X,W,Y,W,W,Y,W,Y,X,Y,W,X,W,W,Y,Z,X,W,W,X,X,Y,W,W,Y,Z,Y,W,X,Z,X,Y,Z,Z,X,Y,X,W,Y,Y,W,Y,X,W,Y,X,Y,X,Y,X,X,W,X,Y,Z,X,Y,X,W,Y,Z,Y,W,Z,X,X,Y,Z,Z,X,Y,Y,W,Y,W,W,Y,W,W,W,X,W,W,Y,X,W,Y,W,Z,X,W,Y,X,Y,W,Y,W,W,Z,X,Y,W,Y,Z,Y,W,X,Y,X,Y,Z,Y,Y,Y,X,Y,X,Y,Y,X,Y,X,Y,X,X,Y,W,Y,X,X,Y,Y,Y,W,W,Z,X,Y,X,Y,Y,Y,W,Y,Y,Y,X,Y,Y,Y,X,Y,Y,X,Z,Y,X,Y,W,Z,X,W,Y,W,Y,Y,Y,Y,Z,Y,Z,X,W,W,X,Y,Z,Z,X,Y,Y,X,Y,Z,X,W,Y,W,Z,Z,W,W,Y,X,X,W,Y,X,W,Y,X,Z,Y,Y,X,X,W,W,Y,W,Y,Z,X,W,Z,W,X,Y,Z,W,Z,Y,Y,W,Z,X,Z,Z,Z,W,W,X,Z,W,X,Z,Z,X,Z,Y,Y,W,W,Z,X,Y,Y,Z,X,Y,Y,X,Z,Y,X,Z,Y,W,Y,X,W,Y,X,Y,X,W,Y,W,W,Y,Y,Y,X,W,W,X,Z,X,Z,X,W,Y,Z,Z,W,Y,Y,Z,X,W,X,W,W,Z,W,W,W,Z,Y,W,Y,X,Z,X,W,W,Y,X,Y,W,Z,X,W,Y,W,Y,Z,Y,Z,W,W,Z,W,W,Y,W,X,W,Y,W,W,W,Y,Y,X,W,W,Y,Z,X,W,Z,W,X,Y,W,Z,Z,Z,Y,Z,Z,Z,Z,X,W,Z,X,W,Y,X,Y,Y,W,Z,X,X,Z,W,X,Y,Y,W,Y,W,W,W,Y,X,W,X,X,X,X,W,X,Z,W,Z,W,W,W,Y,X,Z,X,Y,X,Y,X,Z,X,Z,Z,X,W,Y,X,W,X,X,W,W,Y,W,X,X,W,X,X,Y,X,Y,X,W,Y,X,Y,W,X,X,Y,Z,W,W,X,Y,Y,W,Y,Y,Z,W,Y,X,W,Y,Z,W,X,W,Y,X,W,Z,X,Z,X,Z,X,X,W,W,Y,X,Y,W,W,W,Z,W,Y,X,X,Z,X,X,Y,Z,W,Y,Z,X,Y,Z,X,Y,X,W,W,Y,W,X,Y,Z,X,W,Y,W,Y,W,Y,W,X,W,Y,Y,X,W,W,Y,X,Y,X,Y,Y,X,W,Z,X,W,X,W,X,W,Y,W,X,X,Z,X,W,X,W,Y,Y,W,W,Y,W,W,X,X,Y,W,Z,X,W,Y,Y,W,W,Y,X,W,Y,X,W,Y,W,X,Y,X,W,W,Y,Y,X,X,Y,X,W,Y,X,Z,Y,X,X,Y,X,X,W,W,X,W,X,Y,W,X,Y,Y,W,W,Y,Y,Y,W,Z,X,X,Y,X,X,Z,X,Z,Y,W,X,Y,W,W,W,X,W,Z,Y,Y,Y,Y,Y,Y,W,X,Y,W,Y,X,W,Z,W,W,W,Y,W,Y,Z,Y,W,Z,X,Y,Y,X,X,W,Y,Z,Y,X,Y,Y,X,Z,Z,Z,Y,W,Y,X,Y,X,Y,Z,X,W,W,Y,Y,Z,W,Z,Z,Z,Y,W,X,Z,Z,X,Z,X,W,Z,X,Y,Y,X,Y,W,Y,W,X,X,X,W,Z,X,W,Y,X,W,X,W,Z,X,Z,W,Y,Y,Z,Z,Y,Z,X,W,W,X,Y,W,Y,X,W,Y,W,W,Z,X,W,W,W,Y,W,Z,W,Y,Z,W,Y,W,W,W,Y,X,W,Y,X,W,Z,X,W,W,X,X,Y,X,W,X,W,Y,X,W,X,X,Z,Y,X,X,Y,Y,Y,Y,W,Z,X,W,Z,X,W,X,Y,Y,X,W,Z,Y,X,W,Y,Y,X,X,Y,X,Y,W,W,Z,W,X,Y,Y,W,Y,Y,Z,X,Y,Y,X,Y,Y,Y,Z,Y,X,X,Z,Z,X,W,Y,Z,W,Z,Y,Y,W,Y,X,Y,Y,W,Z,X,Y,Z,X,Y,W,Y,X,X,Y,Y,W,Y,X,W,Z,W,W,Z,W,W,Y,X,W,Y,X,Z,W,X,Y,X,W,Y,W,Y,Y,W,W,X,W,Z,X,W,Z,X,W,Z,W,Z,X,Z,X,W,W,Z,W,Z,W,Y,Y,X,Y,Y,W,Y,W,Y,X,Z,Z,W,Z,X,X,X,Z,W,Z,Y,W,X,W,Y,Y,Y,W,Y,Y,W,Y,X,X,W,Y,Y,Y,X,Z,X,W,Z,Y,X,Y,Y,W,Y,Y,Y,W,Z,Y,X,W,Z,X,W,X,W,Z,X,W,Y,Z,X,W,Z,W,Y,W,Y,X,Y,Z,X,X,Y,Y,Z,Z,W,Y,X,Y,Y,Z,Z,X,X,W,Z,X,W,Y,X,W,Y,Y,W,Z,X,Y,Z,W,W,W,W]

d = Decoder(pi, trans, obs)

output=d.Decode(data_2)

for i in range (0,len(output)):
    output[i] +=1

print("Length of data set: ", end="") 
print (len(output))

for i in range (0,len(output)):
    print(output[i], end="")    

Length of data set: 25
2121321321213213213213213

In [6]:
# this test is partly taken from cuHMM (https://code.google.com/p/chmm/)
# pi = np.array([[0.04, 0.02, 0.06, 0.04, 0.11, 0.11, 0.01, 0.09, 0.03, 0.05, 0.06, 0.11, 0.05, 0.11, 0.03, 0.08]]).T
# trans = np.array([ \
#     [0.08, 0.02, 0.10, 0.05, 0.07, 0.08, 0.07, 0.04, 0.08, 0.10, 0.07, 0.02, 0.01, 0.10, 0.09, 0.01], \
#     [0.06, 0.10, 0.11, 0.01, 0.04, 0.11, 0.04, 0.07, 0.08, 0.10, 0.08, 0.02, 0.09, 0.05, 0.02, 0.02], \
#     [0.08, 0.07, 0.08, 0.07, 0.01, 0.03, 0.10, 0.02, 0.07, 0.03, 0.06, 0.08, 0.03, 0.10, 0.10, 0.08], \
#     [0.08, 0.04, 0.04, 0.05, 0.07, 0.08, 0.01, 0.08, 0.10, 0.07, 0.11, 0.01, 0.05, 0.04, 0.11, 0.06], \
#     [0.03, 0.03, 0.08, 0.10, 0.11, 0.04, 0.06, 0.03, 0.03, 0.08, 0.03, 0.07, 0.10, 0.11, 0.07, 0.03], \
#     [0.02, 0.05, 0.01, 0.09, 0.05, 0.09, 0.05, 0.12, 0.09, 0.07, 0.01, 0.07, 0.05, 0.05, 0.11, 0.06], \
#     [0.11, 0.05, 0.10, 0.07, 0.01, 0.08, 0.05, 0.03, 0.03, 0.10, 0.01, 0.10, 0.08, 0.09, 0.07, 0.02], \
#     [0.03, 0.02, 0.16, 0.01, 0.05, 0.01, 0.14, 0.14, 0.02, 0.05, 0.01, 0.09, 0.07, 0.14, 0.03, 0.01], \
#     [0.01, 0.09, 0.13, 0.01, 0.02, 0.04, 0.05, 0.03, 0.10, 0.05, 0.06, 0.06, 0.11, 0.06, 0.03, 0.14], \
#     [0.09, 0.03, 0.04, 0.05, 0.04, 0.03, 0.12, 0.04, 0.07, 0.02, 0.07, 0.10, 0.11, 0.03, 0.06, 0.09], \
#     [0.09, 0.04, 0.06, 0.06, 0.05, 0.07, 0.05, 0.01, 0.05, 0.10, 0.04, 0.08, 0.05, 0.08, 0.08, 0.10], \
#     [0.07, 0.06, 0.01, 0.07, 0.06, 0.09, 0.01, 0.06, 0.07, 0.07, 0.08, 0.06, 0.01, 0.11, 0.09, 0.05], \
#     [0.03, 0.04, 0.06, 0.06, 0.06, 0.05, 0.02, 0.10, 0.11, 0.07, 0.09, 0.05, 0.05, 0.05, 0.11, 0.08], \
#     [0.04, 0.03, 0.04, 0.09, 0.10, 0.09, 0.08, 0.06, 0.04, 0.07, 0.09, 0.02, 0.05, 0.08, 0.04, 0.09], \
#     [0.05, 0.07, 0.02, 0.08, 0.06, 0.08, 0.05, 0.05, 0.07, 0.06, 0.10, 0.07, 0.03, 0.05, 0.06, 0.10], \
#     [0.11, 0.03, 0.02, 0.11, 0.11, 0.01, 0.02, 0.08, 0.05, 0.08, 0.11, 0.03, 0.02, 0.10, 0.01, 0.11]])
# obs = np.array([[0.01,0.99], \
#                 [0.58,0.42], \
#                 [0.48,0.52], \
#                 [0.58,0.42], \
#                 [0.37,0.63], \
#                 [0.33,0.67], \
#                 [0.51,0.49], \
#                 [0.28,0.72], \
#                 [0.35,0.65], \
#                 [0.61,0.39], \
#                 [0.97,0.03], \
#                 [0.87,0.13], \
#                 [0.46,0.54], \
#                 [0.55,0.45], \
#                 [0.23,0.77], \
#                 [0.76,0.24]])
pi = np.array([[0.32, 0.28, 0.4 ]]).T
trans = np.array([ \
    [0, 0.2, 0.8], \
    [0.7, 0.1, 0.2], \
    [0.3, 0.6, 0.1]])
obs = np.array([[0.6,0.1, 0.2, 0.1], \
                [0.1, 0.7, 0.1, 0.1], \
                [0.2, 0, 0.5, 0.3]])
data = [[0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [1, 1, 0, 0, 1, 1, 1, 0, 0, 0]]
d = Decoder(pi, trans, obs)
# assert d.Decode(data[0]) == [11, 10, 15, 10, 15, 10, 0, 10, 0, 0]
# assert d.Decode(data[1]) == [5, 14, 10, 15, 0, 0, 14, 10, 15, 10]
d.Decode(data[0])
# print ("PASSED")

[0, 2, 0, 2, 0, 2, 1, 0, 1, 0]

In [ ]:
input : XWZYWZXWXW
out   : 2132132121
